# StatsOptimize

## Intro and why Update Stats

While a SqlPool can automatically create statistics in response to new table or operation on a column, it will never update this statistic. This may be ok if the table is only ever loaded once, but if the table has a lot of subsequent data movement, the original statistics will become out of date. We call this an out of date or **_<mark>stale statistic</mark>_**

If a table is empty or has very limited data, SQL Server may also add “generic” stats that assume 1,000 rows, regardless of the actual row size. We call this a <mark>**_missing statistic_**.</mark>

Lastly, we may have auto stats on a single column but ALSO another stat covering that column. This is a waste of resources as the query plan can only use one of the statists. We call that **_<mark>overlapping statistics</mark>_** and the impact of this is just wasted disk space and time maintaining them.

## 2.1 First Things First. Check Auto Stats

You can confirm this with the query below if auto stats is created.

This also shows us a few other things

- If Query Store is on (it probably should be)
- If Result Set Caching is enabled (it probably should be)
- We can see that Auto Update Stats is not enabled (not currently supported on SqlPools)

If auto stats creation is not enabled, then you can enable it with the following command (recommended).

ALTER DATABASE \<yourdatawarehousename\>  
SET AUTO\_CREATE\_STATISTICS ON

More details is contained in MS Article below

[https://docs.microsoft.com/en-us/azure/synapse-analytics/sql/develop-tables-statistics](https://docs.microsoft.com/en-us/azure/synapse-analytics/sql/develop-tables-statistics)

In [1]:
select name, d.is_query_store_on, is_result_set_caching_on, is_query_store_on, is_auto_update_stats_on, is_auto_create_stats_on
from sys.databases d where d.name =db_name()

(1 row affected)

Total execution time: 00:00:00.157

name,is_query_store_on,is_result_set_caching_on,is_query_store_on,is_auto_update_stats_on,is_auto_create_stats_on
AdventureWorksDW,1,1,1,0,1


In [2]:
/* 
    We can see auto stats in action here by creating table . It will just hav estats on the CX

    We have Views that show Stats at three different Levels
    - dbo.vStats
    - dbo.vPartitionStats (you wont need this unless using partitioning at say > 300  billion rows)
*/

IF OBJECT_ID('[dbo].[FactFinance1m]') is not null 
	DROP TABLE [dbo].[FactFinance1m]
GO
CREATE TABLE [dbo].[FactFinance1m] WITH (
	DISTRIBUTION = ROUND_ROBIN, CLUSTERED COLUMNSTORE INDEX 
) AS
SELECT TOP 1000000 * FROM [dbo].[FactFinance1b]
GO

SELECT * FROM dbo.vStats WHERE table_name='FactFinance1m'



Commands completed successfully.

(1000000 rows affected)

(1 row affected)

Total execution time: 00:00:07.822

object_id,stat_name,table_name,schema_name,stats_id,auto_created,filter_definition,last_updated_date,stat_columns,stats_row_count,actual_row_count,stats_difference_percent,sqlCommand
1974350148,ClusteredIndex_18ec512b03c0466ea3b09f61eeaa3710,FactFinance1m,dbo,1,0,NULL,NULL,"DateKey,DepartmentGroupKey,ScenarioKey,OrganizationKey,AccountKey,Amount,Date,LineageKey",1000000,1000000,0.00,UPDATE STATISTICS [dbo].[FactFinance1m] (ClusteredIndex_18ec512b03c0466ea3b09f61eeaa3710) WITH FULLSCAN


If we run a query doing an aggregate on a column we can see the auto stats kick in.

We can also see the the potenital for overlapping stats

In [1]:
SELECT Count(distinct AccountKey) as accountKeys, Count(distinct DateKey) as Dates from [dbo].[FactFinance1m] 

/* Prodata View to View Stats Information */
SELECT * FROM dbo.vStats WHERE table_name='FactFinance1m'

(1 row affected)

(3 rows affected)

Total execution time: 00:00:04.870

accountKeys,Dates
70,38


object_id,stat_name,table_name,schema_name,stats_id,auto_created,filter_definition,last_updated_date,stat_columns,stats_row_count,actual_row_count,stats_difference_percent,stats_sample_rate,sqlCommand
1974350148,ClusteredIndex_18ec512b03c0466ea3b09f61eeaa3710,FactFinance1m,dbo,1,0,NULL,NULL,"DateKey,DepartmentGroupKey,ScenarioKey,OrganizationKey,AccountKey,Amount,Date,LineageKey",1000000,1000000,0.00,100,UPDATE STATISTICS [dbo].[FactFinance1m] (ClusteredIndex_18ec512b03c0466ea3b09f61eeaa3710) WITH FULLSCAN
1974350148,_WA_Sys_00000001_75AE3144,FactFinance1m,dbo,2,1,NULL,2022-07-07 20:10:05.897,DateKey,1000000,1000000,0.00,100,UPDATE STATISTICS [dbo].[FactFinance1m] (_WA_Sys_00000001_75AE3144) WITH FULLSCAN
1974350148,_WA_Sys_00000005_75AE3144,FactFinance1m,dbo,3,1,NULL,2022-07-07 20:10:06.537,AccountKey,1000000,1000000,0.00,100,UPDATE STATISTICS [dbo].[FactFinance1m] (_WA_Sys_00000005_75AE3144) WITH FULLSCAN


In [7]:
/*We can see what a Stats object looks like with below query */


DBCC SHOW_STATISTICS ('dbo.FactFinance1m' , _WA_Sys_00000001_75AE3144)



(1 row affected)

(1 row affected)

(38 rows affected)

Total execution time: 00:00:00.081

Name,Updated,Rows,Rows Sampled,Steps,Density,Average key length,String Index,Filter Expression,Unfiltered Rows,Persisted Sample Percent
_WA_Sys_00000001_75AE3144,Jul 7 2022 8:10PM,1000000,1000000,38,0.02631579,4,NO,NULL,1000000,0


All density,Average Length,Columns
0.032310177,4,DateKey


RANGE_HI_KEY,RANGE_ROWS,EQ_ROWS,DISTINCT_RANGE_ROWS,AVG_RANGE_ROWS
20101229,0,34316,0,1
20110129,0,40046,0,1
20110301,0,39811,0,1
20110331,0,42011,0,1
20110501,0,38851,0,1
20110531,0,42628,0,1
20110701,0,27783,0,1
20110801,0,39243,0,1
20110829,0,37077,0,1
20110929,0,23396,0,1
